# Webcrawler

### Importierte Bibliotheken

Diese Bibliotheken werden verwendet, um HTTP-Anfragen zu senden, HTML-Inhalte zu analysieren, URL-Manipulationen durchzuführen und Threading zu verwenden.

In [ ]:
from bs4 import BeautifulSoup  
from urllib.parse import urljoin, urlparse 
from urllib.robotparser import RobotFileParser  

In [ ]:
import os  
import requests  
import threading  
import time  

Erklärung:
- BeautifulSoup: Werkzeug zum Parsen und Analysieren von HTML- und XML-Dokumenten.
- urljoin, urlparse: Funktionen zur Bearbeitung und Manipulation von URLs.
- RobotFileParser: Klasse zur Analyse und Interpretation der robots.txt-Datei, um die Crawling-Regeln einer Website zu überprüfen.
- os: Modul zur Interaktion mit dem Betriebssystem (z. B. für Dateioperationen).
- requests: Bibliothek zum Senden von HTTP-Anfragen an Webseiten.
- threading: Modul zum Erstellen und Verwalten von Threads für gleichzeitige Operationen.
- time: Modul zur Implementierung von Verzögerungen (z. B. zwischen Anfragen).

### Funktion, um zu prüfen, ob das Crawlen einer URL erlaubt ist

In [ ]:
def can_fetch(url, user_agent='*'):
    parsed_url = urlparse(url) 
    base_url = f"{parsed_url.scheme}://{parsed_url.netloc}" 
    rp = RobotFileParser()  
    rp.set_url(urljoin(base_url, '/robots.txt')) 
    rp.read() 
    return rp.can_fetch(user_agent, url)  

Die Funktion überprüft die `robots.txt`-Datei der Website, um zu sehen, ob der angegebene User-Agent auf die URL zugreifen darf.
Erklärung:
- can_fetch: Funktion zur Überprüfung, ob ein User-Agent eine bestimmte URL gemäß robots.txt crawlen darf.
- urlparse: Teilt die gegebene URL in ihre Komponenten (Schema, Host, Pfad usw.).
- base_url: Erzeugt die Basis-URL, um die robots.txt-Datei zu erreichen.
- RobotFileParser: Ein Objekt, das die Regeln aus robots.txt einliest und anwendet.
- can_fetch: Überprüft, ob die gegebene URL für den angegebenen User-Agent zugänglich ist.

### Hauptfunktion zum Crawlen von Webseiten

In [ ]:
def crawl(url, max_depth=2, delay=1):
    visited = set()  
    lock = threading.Lock()  

  
    def _crawl(current_url, depth):
        
        if depth > max_depth or current_url in visited:
            return

        
        with lock:
            print(f"Crawling: {current_url} at depth {depth}")
            visited.add(current_url)  

        
        if not can_fetch(current_url):
            print(f"Access denied by robots.txt for {current_url}")
            return

        try:
            
            response = requests.get(current_url, timeout=10)
            response.raise_for_status() 
        except requests.RequestException as e:
            
            print(f"Failed to retrieve {current_url}: {e}")
            return

       
        soup = BeautifulSoup(response.text, 'html.parser')

        
        with lock:
            with open("crawled_links.txt", "a") as file:
                file.write(current_url + "\n")  

        
        for link in soup.find_all('a', href=True):
            href = link.get('href')  
            full_url = urljoin(current_url, href)  
            if full_url not in visited:
                time.sleep(delay)  
                _crawl(full_url, depth + 1)  

    
    threads = []
    for i in range(5):  
        t = threading.Thread(target=_crawl, args=(url, 0))
        t.start() 
        threads.append(t)

    
    for t in threads:
        t.join()

Erklärung:
- crawl: Hauptfunktion zum Starten des Web-Crawling-Prozesses.
- visited: Set zur Speicherung bereits besuchter URLs, um Doppelbesuche zu vermeiden.
- lock: Ein Mechanismus zur Gewährleistung der Thread-Sicherheit beim Zugriff auf geteilte Ressourcen.
- _crawl: Rekursive Hilfsfunktion zum Crawlen der gegebenen URL bis zur maximalen Tiefe.
- requests.get: Senden einer HTTP-Anfrage an die aktuelle URL.
- BeautifulSoup: Parsen der HTML-Inhalte zur Extraktion von Links.
- soup.find_all('a', href=True): Sucht alle Anker-Elemente mit einem href-Attribut.
- time.sleep(delay): Fügt eine Verzögerung zwischen den Anfragen hinzu, um die Serverbelastung zu reduzieren.
- threading.Thread: Erzeugt und startet Threads für paralleles Crawlen.

### Benutzerinteraktion und Skriptausführung

In [5]:
start_url = input("Bitte geben Sie die Start-URL ein: ")

if not start_url.startswith(('http://', 'https://')):
    start_url = 'https://' + start_url

max_depth = int(input("Bitte geben Sie die maximale Tiefe ein: "))
delay = float(input("Bitte geben Sie die Verzögerung zwischen den Anfragen in Sekunden ein: "))

if os.path.exists("crawled_links.txt"):
    os.remove("crawled_links.txt")

crawl(start_url, max_depth, delay)

Erklärung:
- input: Fordert den Benutzer zur Eingabe der Start-URL, der maximalen Tiefe und der Verzögerung auf.
- startswith: Überprüft, ob die URL mit http:// oder https:// beginnt, und fügt andernfalls https:// hinzu.
- os.path.exists: Überprüft, ob die Datei crawled_links.txt existiert, und löscht sie, wenn sie existiert.
- crawl: Startet den Crawl-Prozess mit den angegebenen Parametern.